In [1]:
from pygradu import gridify
import shapely.geometry
import pandas as pd
import numpy as np
import importlib

%time grid = gridify.area_to_grid(side_length=5000)

Original NE: POINT (134 40)
Original SW: POINT (123 32)
Transformed NE: (798015.8738997555, 745576.6479352411)
Transformed SW: (-178108.15703831997, -158644.45983641956)
col_count 195
row_count 180
grid_size 35100
CPU times: user 4.06 ms, sys: 191 µs, total: 4.25 ms
Wall time: 4.31 ms


In [2]:
# Import dataset

ais = pd.read_csv('data/datasets/ko_ais_test.csv', parse_dates = ['timestam'])
ais = ais.sort_values(by=['mmsi', 'timestam'])
ais.head()

/home/eunkyu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,mmsi,timestam,lat,lon,speed,course,heading,turnrate,breath,vessel_type,vessel_max_speed,draft,power,dwt,iceclass
2486,2486,0,2019-09-01 00:00:10,33.962062,126.818543,4.5,271.7,278,0,6.0,32,16,0.3,10000,203.0,IA
4218,4218,0,2019-09-01 00:00:18,33.962068,126.818342,4.5,272.2,279,0,6.0,32,16,0.3,10000,203.0,IA
6827,6827,0,2019-09-01 00:00:29,33.962082,126.818090,4.5,280.8,281,0,6.0,32,16,0.3,10000,203.0,IA
9349,9349,0,2019-09-01 00:00:40,33.962108,126.817797,4.4,281.1,284,0,6.0,52,16,0.0,10000,114.0,IA
11486,11486,0,2019-09-01 00:00:50,33.962140,126.817550,4.4,275.2,285,0,6.0,52,16,0.0,10000,114.0,IA


In [3]:
ais.columns=ais.columns.str.replace('timestam', 'timestamp')
ais

,Unnamed: 0,mmsi,timestamp,lat,lon,speed,course,heading,turnrate,breath,vessel_type,vessel_max_speed,draft,power,dwt,iceclass
2486,2486,0,2019-09-01 00:00:10,33.962062,126.818543,4.5,271.7,278,0,6.0,32,16,0.3,10000,203.0,IA
4218,4218,0,2019-09-01 00:00:18,33.962068,126.818342,4.5,272.2,279,0,6.0,32,16,0.3,10000,203.0,IA
6827,6827,0,2019-09-01 00:00:29,33.962082,126.818090,4.5,280.8,281,0,6.0,32,16,0.3,10000,203.0,IA
9349,9349,0,2019-09-01 00:00:40,33.962108,126.817797,4.4,281.1,284,0,6.0,52,16,0.0,10000,114.0,IA
11486,11486,0,2019-09-01 00:00:50,33.962140,126.817550,4.4,275.2,285,0,6.0,52,16,0.0,10000,114.0,IA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21597851,21597851,999999999,2019-09-01 23:34:50,35.444632,129.365895,0.0,128.0,132,0,NaN,NaN,16,NaN,10000,NaN,IA
21748281,21748281,999999999,2019-09-01 23:45:50,35.444642,129.365903,0.0,109.1,131,0,NaN,NaN,16,NaN,10000,NaN,IA
21806599,21806599,999999999,2019-09-01 23:50:06,33.521342,126.360397,1.7,252.2,511,0,NaN,NaN,16,NaN,10000,NaN,IA
21888703,21888703,999999999,2019-09-01 23:56:07,33.520707,126.357025,1.9,242.2,511,0,NaN,NaN,16,NaN,10000,NaN,IA


In [4]:
ais = ais[ais.speed > 0.1]

In [5]:
# Remove unnecessary columns
%time ais_subset = ais[['timestamp', 'mmsi', 'lat', 'lon', 'speed', 'vessel_type']]

CPU times: user 158 ms, sys: 71.8 ms, total: 230 ms
Wall time: 227 ms


In [6]:
%time ais_subset['grid_point'] = ais_subset.apply(lambda row: grid.get_grid_point(row.lat, row.lon), axis=1)

CPU times: user 3min 50s, sys: 2.6 s, total: 3min 53s
Wall time: 3min 53s


/home/eunkyu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
%time ais_subset['node'] = ais_subset.apply(lambda row: grid.get_grid_position(row), axis=1)

CPU times: user 3min 14s, sys: 4.75 s, total: 3min 19s
Wall time: 3min 16s


/home/eunkyu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
importlib.reload(gridify)

%time avg_speeds = gridify.get_avg_speeds(ais_subset)
avg_speeds.head()

CPU times: user 38.4 s, sys: 194 ms, total: 38.5 s
Wall time: 38.6 s


,2375,2376,2377,2378,2379,2380,2381,2382,2383,2384,...,25707,25708,25713,25714,25715,25716,25887,25894,25895,25896
0,16.050000,11.040000,8.95,6.800,8.066667,7.406250,1.372917,6.195833,2.015075,2.137811,...,14.700000,6.600,NaN,11.950000,13.17500,11.5,16.0,12.46,13.0,NaN
30,15.633333,16.416667,9.35,13.925,8.150000,7.153061,2.908629,4.729825,1.945128,2.003343,...,11.679310,6.625,NaN,11.926087,13.10625,11.5,NaN,12.40,13.0,NaN
52,12.788889,8.700000,NaN,9.760,8.300000,7.290000,1.707143,7.366667,2.373077,3.226220,...,11.866667,NaN,NaN,NaN,13.10000,NaN,NaN,NaN,13.0,NaN
70,8.550000,20.550000,NaN,7.000,8.100000,7.411111,2.346809,3.518750,1.546667,2.621569,...,9.838462,NaN,NaN,11.733333,NaN,11.5,NaN,NaN,13.0,NaN
90,18.500000,NaN,NaN,NaN,NaN,NaN,0.457143,NaN,1.660000,1.142857,...,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
avg_speeds

,2375,2376,2377,2378,2379,2380,2381,2382,2383,2384,...,25707,25708,25713,25714,25715,25716,25887,25894,25895,25896
0,16.050000,11.040000,8.95,6.800,8.066667,7.406250,1.372917,6.195833,2.015075,2.137811,...,14.700000,6.600,NaN,11.950000,13.17500,11.5,16.0,12.46,13.0,NaN
30,15.633333,16.416667,9.35,13.925,8.150000,7.153061,2.908629,4.729825,1.945128,2.003343,...,11.679310,6.625,NaN,11.926087,13.10625,11.5,NaN,12.40,13.0,NaN
52,12.788889,8.700000,NaN,9.760,8.300000,7.290000,1.707143,7.366667,2.373077,3.226220,...,11.866667,NaN,NaN,NaN,13.10000,NaN,NaN,NaN,13.0,NaN
70,8.550000,20.550000,NaN,7.000,8.100000,7.411111,2.346809,3.518750,1.546667,2.621569,...,9.838462,NaN,NaN,11.733333,NaN,11.5,NaN,NaN,13.0,NaN
90,18.500000,NaN,NaN,NaN,NaN,NaN,0.457143,NaN,1.660000,1.142857,...,6.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
avg_speeds.to_dict()

{2375: {0: 16.05,
  30: 15.633333333333335,
  52: 12.78888888888889,
  70: 8.55,
  90: 18.5,
  55: nan,
  60: nan,
  80: nan,
  192: nan,
  32: nan,
  37: nan,
  56: nan,
  71: nan,
  82: nan,
  50: nan,
  51: nan,
  67: nan,
  155: nan,
  40: nan,
  64: nan,
  69: nan,
  73: nan,
  81: nan,
  84: nan,
  95: nan,
  98: nan,
  99: nan,
  248: nan,
  253: nan,
  3: nan,
  20: nan,
  29: nan,
  31: nan,
  36: nan,
  53: nan,
  54: nan,
  68: nan,
  72: nan,
  74: nan,
  75: nan,
  79: nan,
  83: nan,
  88: nan,
  89: nan,
  136: nan,
  151: nan,
  185: nan,
  196: nan,
  204: nan,
  247: nan,
  6: nan,
  62: nan,
  132: nan,
  140: nan,
  153: nan,
  10: nan,
  24: nan,
  28: nan,
  65: nan,
  85: nan,
  122: nan,
  184: nan,
  197: nan,
  219: nan,
  243: nan,
  254: nan,
  '0': nan,
  '23': nan,
  '30': nan,
  '32': nan,
  '37': nan,
  1: nan,
  11: nan,
  33: nan,
  34: nan,
  49: nan,
  76: nan,
  77: nan,
  92: nan,
  124: nan,
  142: nan,
  152: nan,
  157: nan,
  200: nan,
  201: n

In [11]:
avg_speeds.to_csv('data/models/speed_5km2.csv')